# COGS 188 - Project Proposal

# Project Description

You have the choice of doing either (1) an AI solve a problem style project or (2) run a Special Topics class on a topic of your choice.  If you want to do (2) you should fill out the _other_ proposal for that. This is the proposal description for (1).

You will design and execute a machine learning project. There are a few constraints on the nature of the allowed project. 
- The problem addressed will not be a "toy problem" or "common training students problem" like 8-Queens or a small Traveling Salesman Problem or similar
- If its the kind of problem (e.g., RL) that interacts with a simulator or live task, then the problem will have a reasonably complex action space. For instance, a wupus world kind of thing with a 9x9 grid is definitely too small.  A simulated mountain car with a less complex 2-d road and simplified dynamics seems like a fairly low achievement level.  A more complex 3-d mountain car simulation with large extent and realistic dynamics, sure sounds great!
- If its the kind of problem that uses a dataset, then the dataset will have >1k observations and >5 variables. I'd prefer more like >10k observations and >10 variables. A general rule is that if you have >100x more observations than variables, your solution will likely generalize a lot better. The goal of training an unsupervised machine learning model is to learn the underlying pattern in a dataset in order to generalize well to unseen data, so choosing a large dataset is very important.
- The project must include some elements we talked about in the course
- The project will include a model selection and/or feature selection component where you will be looking for the best setup to maximize the performance of your AI system. Generally RL tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible. 
- You will evaluate the performance of your AI system using more than one appropriate metric
- You will be writing a report describing and discussing these accomplishments


Feel free to delete this description section when you hand in your proposal.

# Names

- Sebastian Modafferi
- Chirag Amatya
- Michael Chu

# Abstract 

The goal of this project is to develop a reinforcement learning agent capable of successfully landing a spacecraft in the Lunar Lander environment provided by Gymnasium. This environment simulates a classic rocket trajectory optimization problem, controlling of the lander’s engines to achieve a safe landing. The RL agent will use the 8-dimensional state space, consisting of position, velocity, orientation, and ground contact indicators, to select discrete actions such as firing the main or orientation engines. A reward-based feedback system will guide the agent, encouraging successful landings while penalizing crashes and excessive fuel usage. The RL agent will process the data to learn an optimal policy for controlling the spacecraft through iterative training. Success will be measured using metrics such as average total reward per episode, success rate (percentage of safe landings), and fuel efficiency.

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

The problem we are solving is the [lunar lander task](https://gymnasium.farama.org/environments/box2d/lunar_lander/) provided by the `gymnasium` python package. The task is to land a vehicle powered by three engines onto a randomly-generated surface. The lander is comrpised of two orientation engines, which add angular velocity to rotate the lander, along with one main engine which provides x/y-velocity depending on the orientation. The task begins with the lander in the top-center of the simulation, with a random initial force acting upon it. The lander must then use the three engines to not only correct for the initial force, but to also touchdown near (0,0), such that the two "legs" contact the surface. In order to successfully complete the task the lander must land close to (0,0) and must use the engines sparingly, as their is a reward cost for engine usage. The total reward breakdown is listed below, as described on [the docs](https://gymnasium.farama.org/environments/box2d/lunar_lander/).


- **Increases** the closer the lander is to the landing pad and **decreases** the further it is.
- **Increases** the slower the lander is moving and **decreases** the faster it is moving.
- **Decreases** the more the lander is tilted (i.e., the angle is not horizontal).
- **Increases** by **10 points** for each leg that is in contact with the ground.
- **Decreases** by **0.03 points** for each frame a side engine is firing.
- **Decreases** by **0.3 points** for each frame the main engine is firing.

Additionally, the episode receives:

- **-100 points** for crashing.
- **+100 points** for landing safely.

In order to successfully complete the task, the lander must achieve __at least__ 200 points. Additional points will be used to compare performance across multiple agents.

# Data

For the Lunar Lander environment from Gymnasium, the data consists of simulated episodes of landing attempts. The dataset is dynamically generated using the environment and represents the lander’s state transitions as it interacts with the environment. Due to the nature of training reinforcement learning models, no dataset will be collected or manufactured from the simulation Instead, the agents will learn via exploration and repeated simulations.

- **Source**: Gymnasium Lunar Lander environment (`gymnasium.make("LunarLander-v3")`).
- **Size**: The exact number of episodes required for training will vary by model, however we are looking to benchmark progress after 10,000 episodes of training. This benchmark will then compute the average performance over 100 simulations.
- **Observation Space**:
  - An 8-dimensional state vector:
    1. Horizontal position ($x$): Range $[-2.5, 2.5]$
    2. Vertical position ($y$): Range $[-2.5, 2.5]$
    3. Horizontal velocity ($v_x$): Range $[-10, 10]$
    4. Vertical velocity ($v_y$): Range $[-10, 10]$
    5. Angle ($\theta$): Range $[-6.283, 6.283]$ radians
    6. Angular velocity ($\omega$): Range $[-10, 10]$
    7. Left leg ground contact ($boolean$): 0 (not in contact) or 1 (in contact)
    8. Right leg ground contact ($boolean$): 0 (not in contact) or 1 (in contact)
  - The observation values are floating-point numbers ($float32$).

- **Action Space**: 
  - A set of 4 possible actions:
    1. 0: Do nothing
    2. 1: Fire the left orientation engine
    3. 2: Fire the main engine
    4. 3: Fire the right orientation engine

- **Reward Function**:
  - Positive reward for proximity to the landing pad and stable velocity.
  - $+10$ points for each leg in contact with the ground.
  - $-0.03$ points per frame for using side thrusters.
  - $-0.3$ points per frame for using the main engine.
  - Additional $+100$ points for a successful landing.
  - Additional $-100$ points for a crash.

- **Starting State**:
  - Lander starts at the top center of the viewport with randomized initial force applied to its center of mass.

#### Data Handling (WIP/Not Final):
- **Transformations**:
  - Normalize continuous state variables to the range $[0, 1]$ for better convergence.
  - One-hot encode discrete actions for compatibility with certain ML models

# Proposed Solution

The agent will be a genetically-trained neural network (inspiration<a name="gtnn"></a>[<sup>[x]</sup>](#gtnn)). The exact nueral network structure will remain consisent across generations. However the weights, biases, and connections will be trained through mating and mutating the highest performing members of the population. The exact neuron and layer counts will be hyperparameter optimized for. This will be an effective solution because neural networks are well suited for agent-based tasks, specifically where the agent has a discrete set of actions to take in an environment. Through many generations of breeding and mutating the highest performing genomes, the agent will increasingly perform better and better. Genome-based learning is essential for this implementation due to the lack of a dataset. Using a reinforecement-learning approach will remove the need for us to manufacture a dataset from the environment -- instead allowing the agent to explore the action-observation space relationship.

In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Why might your solution work? Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

To evaluate the performance of the model, the following metrics will be used:

#### Average Total Reward Per Episode
- Measures the cumulative reward collected over an episode.
- Higher rewards indicate better lander behavior.
- **Formula**:

  $R_{\text{total}} = \sum_{t=1}^{T} r_t$

  where $r_t$ is the reward at timestep $t$, and $T$ is the total number of timesteps.
- This metric will be used during model training and will be the primary mode of evaluation.

#### Success Rate
- Percentage of episodes where the lander safely lands on the designated landing pad.
- **Formula**:

  $\text{Success Rate} = \frac{\text{Number of Successful Landings}}{\text{Total Episodes}} \times 100$

#### Fuel Efficiency
- Average fuel usage across episodes. Lower values indicate more efficient lander behavior.
- **Formula**:

  $\text{Fuel Efficiency} = \frac{\text{Total Fuel Consumed}}{\text{Number of Episodes}}$

# Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination. Get creative!

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...
 - Communicate delays or updates on progress as soon as possible.
 - Attend meetings on time - or notify of delays prior.
 - Be respectful of other's opinions.
 - Complete your assigned work with integrity and to your best ability.
 - Share bugs that could be common across other tasks - improve eachother's ability to work efficiently.

# Project Timeline Proposal

Replace this with something meaningful that is appropriate for your needs. It doesn't have to be something that fits this format.  It doesn't have to be set in stone... "no battle plan survives contact with the enemy". But you need a battle plan nonetheless, and you need to keep it updated so you understand what you are trying to accomplish, who's responsible for what, and what the expected due dates are for each item.

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 2/11  |  2 PM | Brainstorm topics/questions (all)  | Communication methods - Project Task - Separation of work for Proposal | 
| 2/13  |  6 PM | Completed proposal assigned sections | discuss proposal submission |
| 2/18  |  2 PM | Decide personal preferences for different approaches | Assign each group member to an approach - Finalize timeline | 
| 2/25  | 2 PM  | boilerplate for training code complete | Discuss goals with model training |
| 2/14  | 2 PM  | First agents should be complete | Discuss hyper-parameter optimization across different approaches |
| 2/23  | 2 PM  | Final agents complete | Discuss code tyding, exporting large functions to .py files for a more readable notebook. |
| 3/4  | 2 PM  | Completion for code-side of Project | Discuss writing strategies - compare preformances |
| 3/11 | 2 PM | Completion of final project submission | Go over the final submission as a group, prepare to submit. |
| 3/19  | Before 11:59 PM  | Whole Project! | Turn in Final Project  |

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem <br>
<a name="gtnn"></a>x.[^](#gtnn): Training a Neural Network using Genetic Algorithm (22 May 2020) [Pezzza's Work](https://www.youtube.com/watch?v=I_p-yfdyP1A).